# Extracting data from KB-labb's API
This script shows an example of extracting text from KB-labb's API and saving it into XML files with some metadata.

In [ ]:
from kblab import Archive
from json import load
import re
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [ ]:
a = Archive('https://betalab.kb.se/', auth=('username', 'password'))

In [ ]:
for package_id in a.search({'label: "GÖTEBORGSPOSTEN", "meta.created":2018'}):
    
    p = a.get(package_id)
    meta = load(p.get_raw('meta.json'))
    
    title = re.search(r"\D+(?= \d)", meta["title"]).group()
    date = re.search(r"(?<=\D )[\d-]+", meta["title"]).group()
    
    root = ET.Element('document')
    child1 = ET.SubElement(root, 'text', attrib={'title':title, 'date':date})
    
    prev_page = "1"
    text = ''
     
    for part in load(p.get_raw('content.json')):
        part_id = part.get('@id', '')
        page_nr = re.search(r"(?<=#[0-9]-)[0-9]{1,2}", part_id).group()
        if prev_page == page_nr:
            text = text + ' ' + (part.get('content', ''))
        else:
            child1_1 = ET.SubElement(child1, 'page', attrib={'nr':prev_page})
            child1_1.text = text
            text = (part.get('content', ''))
            prev_page = page_nr
            
    t = minidom.parseString(ET.tostring(root)).toprettyxml()
    
    with open(f"newspaper_corpora/göteborgsposten_2018/source/{package_id}.xml", "w") as f:
        f.write(t)